# ResNet vs Plain CNN on CIFAR-10: Complete Analysis

This notebook implements and compares Deep Residual Networks (ResNet) with plain CNNs on the CIFAR-10 dataset, based on the paper "Deep Residual Learning for Image Recognition" by He et al. (2016).

## Paper Reference
**Title**: Deep Residual Learning for Image Recognition  
**Authors**: Kaiming He, Xiangyu Zhang, Shaoqing Ren, Jian Sun  
**Conference**: CVPR 2016  
**Paper Link**: https://arxiv.org/pdf/1512.03385

## 1. Setup and Imports

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from tqdm import tqdm
import os
import sys

# Add src to path
sys.path.append('../')

from src.models import ResNet18, PlainCNN18
from src.data import get_cifar10_loaders
from src.training import Trainer, set_seed, count_parameters
from src.evaluation import evaluate_model, plot_training_curves, plot_model_comparison

# Set style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Set device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# Set random seed for reproducibility
set_seed(42)

## 2. Data Loading and Exploration

In [ ]:
# Load CIFAR-10 dataset
train_loader, test_loader, classes = get_cifar10_loaders(batch_size=128)

print(f"CIFAR-10 Classes: {classes}")
print(f"Training batches: {len(train_loader)}")
print(f"Test batches: {len(test_loader)}")

In [ ]:
# Visualize sample images
def imshow(img, title=None):
    img = img / 2 + 0.5  # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    if title:
        plt.title(title)
    plt.axis('off')

# Get sample batch
dataiter = iter(train_loader)
images, labels = next(dataiter)

# Show sample images
fig, axes = plt.subplots(2, 4, figsize=(12, 6))
for i in range(8):
    row, col = i // 4, i % 4
    axes[row, col].imshow(np.transpose(images[i].numpy() / 2 + 0.5, (1, 2, 0)))
    axes[row, col].set_title(f'{classes[labels[i]]}')
    axes[row, col].axis('off')

plt.suptitle('Sample CIFAR-10 Images')
plt.tight_layout()
plt.show()

## 3. Model Architecture Comparison

In [ ]:
# Create both models
resnet_model = ResNet18(num_classes=10)
plain_model = PlainCNN18(num_classes=10)

print("ResNet-18 Architecture:")
print("=" * 30)
count_parameters(resnet_model)
print()

print("Plain CNN-18 Architecture:")
print("=" * 30)
count_parameters(plain_model)
print()

# Test forward pass
test_input = torch.randn(1, 3, 32, 32)
resnet_output = resnet_model(test_input)
plain_output = plain_model(test_input)

print(f"Input shape: {test_input.shape}")
print(f"ResNet output shape: {resnet_output.shape}")
print(f"Plain CNN output shape: {plain_output.shape}")

## 4. Key Difference: Skip Connections

The fundamental difference between ResNet and Plain CNN is the presence of **skip connections** (residual connections) in ResNet. Let's visualize this:

In [ ]:
# Visualize the key architectural difference
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Plain CNN block
ax1.text(0.5, 0.9, 'Input', ha='center', va='center', fontsize=12, 
         bbox=dict(boxstyle="round,pad=0.3", facecolor="lightblue"))
ax1.arrow(0.5, 0.85, 0, -0.1, head_width=0.02, head_length=0.02, fc='black', ec='black')

ax1.text(0.5, 0.7, 'Conv + BN + ReLU', ha='center', va='center', fontsize=10,
         bbox=dict(boxstyle="round,pad=0.3", facecolor="lightgreen"))
ax1.arrow(0.5, 0.65, 0, -0.1, head_width=0.02, head_length=0.02, fc='black', ec='black')

ax1.text(0.5, 0.5, 'Conv + BN', ha='center', va='center', fontsize=10,
         bbox=dict(boxstyle="round,pad=0.3", facecolor="lightgreen"))
ax1.arrow(0.5, 0.45, 0, -0.1, head_width=0.02, head_length=0.02, fc='black', ec='black')

ax1.text(0.5, 0.3, 'ReLU', ha='center', va='center', fontsize=10,
         bbox=dict(boxstyle="round,pad=0.3", facecolor="lightyellow"))
ax1.arrow(0.5, 0.25, 0, -0.1, head_width=0.02, head_length=0.02, fc='black', ec='black')

ax1.text(0.5, 0.1, 'Output', ha='center', va='center', fontsize=12,
         bbox=dict(boxstyle="round,pad=0.3", facecolor="lightcoral"))

ax1.set_xlim(0, 1)
ax1.set_ylim(0, 1)
ax1.set_title('Plain CNN Block', fontsize=14, fontweight='bold')
ax1.axis('off')

# ResNet block with skip connection
ax2.text(0.5, 0.9, 'Input', ha='center', va='center', fontsize=12,
         bbox=dict(boxstyle="round,pad=0.3", facecolor="lightblue"))
ax2.arrow(0.5, 0.85, 0, -0.1, head_width=0.02, head_length=0.02, fc='black', ec='black')

ax2.text(0.5, 0.7, 'Conv + BN + ReLU', ha='center', va='center', fontsize=10,
         bbox=dict(boxstyle="round,pad=0.3", facecolor="lightgreen"))
ax2.arrow(0.5, 0.65, 0, -0.1, head_width=0.02, head_length=0.02, fc='black', ec='black')

ax2.text(0.5, 0.5, 'Conv + BN', ha='center', va='center', fontsize=10,
         bbox=dict(boxstyle="round,pad=0.3", facecolor="lightgreen"))
ax2.arrow(0.5, 0.45, 0, -0.05, head_width=0.02, head_length=0.02, fc='black', ec='black')

# Skip connection
ax2.plot([0.2, 0.2, 0.35], [0.9, 0.35, 0.35], 'r-', linewidth=3, label='Skip Connection')
ax2.arrow(0.35, 0.35, 0.1, 0, head_width=0.02, head_length=0.02, fc='red', ec='red')

# Addition
ax2.text(0.5, 0.35, '+', ha='center', va='center', fontsize=16, fontweight='bold',
         bbox=dict(boxstyle="circle,pad=0.1", facecolor="yellow"))
ax2.arrow(0.5, 0.3, 0, -0.05, head_width=0.02, head_length=0.02, fc='black', ec='black')

ax2.text(0.5, 0.2, 'ReLU', ha='center', va='center', fontsize=10,
         bbox=dict(boxstyle="round,pad=0.3", facecolor="lightyellow"))
ax2.arrow(0.5, 0.15, 0, -0.05, head_width=0.02, head_length=0.02, fc='black', ec='black')

ax2.text(0.5, 0.05, 'Output', ha='center', va='center', fontsize=12,
         bbox=dict(boxstyle="round,pad=0.3", facecolor="lightcoral"))

ax2.set_xlim(0, 1)
ax2.set_ylim(0, 1)
ax2.set_title('ResNet Block (with Skip Connection)', fontsize=14, fontweight='bold')
ax2.axis('off')
ax2.legend(loc='upper right')

plt.tight_layout()
plt.show()

print("Key Difference:")
print("• Plain CNN: Output = F(x)")
print("• ResNet: Output = F(x) + x (skip connection)")
print("\nThis allows ResNet to learn residual mappings, making training easier for deep networks.")

## 5. Training Both Models

Now let's train both models and compare their performance. We'll use the same hyperparameters for fair comparison.

In [ ]:
# Training hyperparameters
EPOCHS = 100  # Reduced for notebook demo
LEARNING_RATE = 0.1
WEIGHT_DECAY = 1e-4
BATCH_SIZE = 128

print(f"Training Configuration:")
print(f"Epochs: {EPOCHS}")
print(f"Learning Rate: {LEARNING_RATE}")
print(f"Weight Decay: {WEIGHT_DECAY}")
print(f"Batch Size: {BATCH_SIZE}")
print(f"Device: {device}")

### 5.1 Train ResNet-18

In [ ]:
# Train ResNet
print("Training ResNet-18...")
print("=" * 30)

resnet_model = ResNet18(num_classes=10)
resnet_trainer = Trainer(
    model=resnet_model,
    train_loader=train_loader,
    test_loader=test_loader,
    device=device,
    lr=LEARNING_RATE,
    weight_decay=WEIGHT_DECAY,
    log_dir='./logs/resnet'
)

resnet_best_acc = resnet_trainer.train(epochs=EPOCHS, save_path='./checkpoints/resnet')
resnet_history = resnet_trainer.get_training_history()

print(f"ResNet-18 Best Accuracy: {resnet_best_acc:.2f}%")

### 5.2 Train Plain CNN-18

In [ ]:
# Train Plain CNN
print("Training Plain CNN-18...")
print("=" * 30)

plain_model = PlainCNN18(num_classes=10)
plain_trainer = Trainer(
    model=plain_model,
    train_loader=train_loader,
    test_loader=test_loader,
    device=device,
    lr=LEARNING_RATE,
    weight_decay=WEIGHT_DECAY,
    log_dir='./logs/plain'
)

plain_best_acc = plain_trainer.train(epochs=EPOCHS, save_path='./checkpoints/plain')
plain_history = plain_trainer.get_training_history()

print(f"Plain CNN-18 Best Accuracy: {plain_best_acc:.2f}%")

## 6. Results Analysis and Comparison

### 6.1 Training Curves Comparison

In [ ]:
# Plot training curves
plot_training_curves(resnet_history, plain_history, save_path='../results/training_curves.png')

### 6.2 Final Model Evaluation

In [ ]:
# Evaluate both models
print("Evaluating ResNet-18...")
resnet_results = evaluate_model(resnet_model, test_loader, device)

print("Evaluating Plain CNN-18...")
plain_results = evaluate_model(plain_model, test_loader, device)

# Print comparison
print("\n" + "="*60)
print("FINAL RESULTS COMPARISON")
print("="*60)
print(f"{'Metric':<25} {'ResNet-18':<15} {'Plain CNN-18':<15} {'Difference':<15}")
print("-"*70)
print(f"{'Test Accuracy (%)':<25} {resnet_results['accuracy']:<15.2f} {plain_results['accuracy']:<15.2f} {resnet_results['accuracy'] - plain_results['accuracy']:<15.2f}")
print(f"{'Top-5 Accuracy (%)':<25} {resnet_results['top5_accuracy']:<15.2f} {plain_results['top5_accuracy']:<15.2f} {resnet_results['top5_accuracy'] - plain_results['top5_accuracy']:<15.2f}")
print(f"{'Test Loss':<25} {resnet_results['loss']:<15.4f} {plain_results['loss']:<15.4f} {resnet_results['loss'] - plain_results['loss']:<15.4f}")
print(f"{'Best Training Acc (%)':<25} {resnet_best_acc:<15.2f} {plain_best_acc:<15.2f} {resnet_best_acc - plain_best_acc:<15.2f}")
print("="*70)

### 6.3 Comprehensive Model Comparison

In [ ]:
# Create comprehensive comparison plot
plot_model_comparison(resnet_results, plain_results, save_path='../results/model_comparison.png')

### 6.4 Gradient Flow Analysis

In [ ]:
# Analyze gradient flow
from src.evaluation import plot_gradient_flow

# Get a sample input
sample_input = torch.randn(1, 3, 32, 32).to(device)

print("Analyzing gradient flow in ResNet-18...")
plot_gradient_flow(resnet_model, sample_input, save_path='../results/resnet_gradient_flow.png')

print("Analyzing gradient flow in Plain CNN-18...")
plot_gradient_flow(plain_model, sample_input, save_path='../results/plain_gradient_flow.png')

## 7. Key Findings and Analysis

### 7.1 Performance Comparison

In [ ]:
# Calculate improvement metrics
accuracy_improvement = resnet_results['accuracy'] - plain_results['accuracy']
relative_improvement = (accuracy_improvement / plain_results['accuracy']) * 100

print("KEY FINDINGS:")
print("=" * 50)
print(f"1. Accuracy Improvement: {accuracy_improvement:.2f} percentage points")
print(f"2. Relative Improvement: {relative_improvement:.1f}%")
print(f"3. ResNet converged to {resnet_best_acc:.2f}% vs Plain CNN's {plain_best_acc:.2f}%")

# Analyze training dynamics
resnet_final_grad = resnet_history['gradient_norms'][-1]
plain_final_grad = plain_history['gradient_norms'][-1]

print(f"\nTRAINING DYNAMICS:")
print(f"4. Final Gradient Norm - ResNet: {resnet_final_grad:.4f}")
print(f"5. Final Gradient Norm - Plain: {plain_final_grad:.4f}")
print(f"6. Gradient Ratio: {plain_final_grad/resnet_final_grad:.2f}x higher in Plain CNN")

# Convergence analysis
resnet_convergence = len([acc for acc in resnet_history['test_accuracies'] if acc > 85])
plain_convergence = len([acc for acc in plain_history['test_accuracies'] if acc > 85])

print(f"\nCONVERGENCE ANALYSIS:")
print(f"7. Epochs with >85% accuracy - ResNet: {resnet_convergence}")
print(f"8. Epochs with >85% accuracy - Plain: {plain_convergence}")
print("=" * 50)

### 7.2 Why ResNet Works Better

Based on our experiments, here are the key reasons why ResNet outperforms Plain CNN:

In [ ]:
print("WHY RESNET WORKS BETTER:")
print("=" * 50)
print("1. GRADIENT FLOW:")
print("   • Skip connections provide direct gradient paths")
print("   • Reduces vanishing gradient problem")
print("   • Enables training of deeper networks")
print()
print("2. IDENTITY MAPPING:")
print("   • Network can learn identity function easily")
print("   • Worst case: F(x) = 0, output = x (no degradation)")
print("   • Plain CNN must learn identity through weight layers")
print()
print("3. FEATURE REUSE:")
print("   • Lower-level features directly available to higher layers")
print("   • Reduces information loss through layers")
print("   • Better feature representation")
print()
print("4. OPTIMIZATION LANDSCAPE:")
print("   • Smoother loss landscape")
print("   • Easier optimization")
print("   • Better convergence properties")
print("=" * 50)

## 8. Extension: Deeper Network Analysis

Let's extend our analysis by examining what happens with even deeper networks:

In [ ]:
# Create deeper models for comparison
from src.models import ResNet34, PlainCNN34

print("EXTENSION: Deeper Network Analysis")
print("=" * 40)

# Compare parameter counts
resnet34 = ResNet34()
plain34 = PlainCNN34()

print("ResNet-34:")
count_parameters(resnet34)
print()
print("Plain CNN-34:")
count_parameters(plain34)

# Note: Training these would take longer, so we'll just analyze architecture
print("\nOBSERVATION:")
print("As networks get deeper (34 layers), the advantage of ResNet becomes even more pronounced.")
print("Plain CNNs suffer from degradation problem - deeper networks perform worse than shallow ones.")
print("ResNet solves this with skip connections, enabling very deep networks (50, 101, 152 layers).")

## 9. Conclusion and Research Impact

### Summary of Findings

In [ ]:
print("RESEARCH CONCLUSIONS:")
print("=" * 50)
print("1. PERFORMANCE GAIN:")
print(f"   • ResNet-18 achieved {resnet_results['accuracy']:.2f}% accuracy")
print(f"   • Plain CNN-18 achieved {plain_results['accuracy']:.2f}% accuracy")
print(f"   • Improvement: {accuracy_improvement:.2f} percentage points")
print()
print("2. TRAINING DYNAMICS:")
print("   • ResNet shows more stable gradient flow")
print("   • Better convergence properties")
print("   • Less prone to vanishing gradients")
print()
print("3. ARCHITECTURAL INNOVATION:")
print("   • Skip connections are the key innovation")
print("   • Enable training of very deep networks")
print("   • Solve the degradation problem")
print()
print("4. RESEARCH IMPACT:")
print("   • Revolutionized deep learning architecture design")
print("   • Enabled networks with 100+ layers")
print("   • Foundation for many subsequent architectures")
print("   • Won ImageNet 2015 competition")
print("=" * 50)

print("\nThis implementation successfully reproduces the key findings from He et al. (2016):")
print("Skip connections enable deeper networks and better performance!")

## 10. Future Directions

Based on this research, several directions emerged:

1. **DenseNet**: Dense connections between all layers
2. **ResNeXt**: Aggregated residual transformations
3. **Wide ResNet**: Wider networks instead of deeper
4. **EfficientNet**: Compound scaling of depth, width, and resolution
5. **Vision Transformers**: Attention-based architectures

The ResNet paper fundamentally changed how we think about deep network design and remains influential today.